In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from numpy import arange
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('trucking_data.csv', index_col = 'year')
df.head()

,lbr_prod,lbr_prod_pctchg,lbr_output,lbr_output_pctchg,emp_trk_trans,emp_cour_msg,emp_drv_sls,est_total,regist,freight_val,...,fuel_avg_per_unit,fuel_avg_mpg,fatalities,fatality_rate_permi,fatality_rate_pertruck,vehicle_inv_permi,vehicle_inv_pertruck,miles_per_emp,miles_per_est,emp_per_est
year,,,,,,,,,,,,,,,,,,,,,
1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64756.0,4586487.0,NaN,...,2467.0,5.5,NaN,NaN,NaN,NaN,NaN,NaN,9.607604e+11,NaN
1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69796.0,5790653.0,NaN,...,3447.0,5.4,1262.0,1.16,2.18,4.96,9.29,NaN,1.554401e+12,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['lbr_prod', 'lbr_prod_pctchg', 'lbr_output', 'lbr_output_pctchg',
       'emp_trk_trans', 'emp_cour_msg', 'emp_drv_sls', 'est_total', 'regist',
       'freight_val', 'freight_val_pctchg', 'freight_val_pcttotal',
       'freight_tons', 'freight_tons_pctchg', 'freight_tons_pcttotal',
       'freight_tonmi', 'freight_tonmi_pctchg', 'freight_tonmi_pcttotal',
       'cptm', 'tax', 'oper_rev', 'oper_exp', 'miles_total',
       'miles_avg_per_unit', 'fuel_total', 'fuel_avg_per_unit', 'fuel_avg_mpg',
       'fatalities', 'fatality_rate_permi', 'fatality_rate_pertruck',
       'vehicle_inv_permi', 'vehicle_inv_pertruck', 'miles_per_emp',
       'miles_per_est', 'emp_per_est'],
      dtype='object')

In [ ]:
df.corr()

,lbr_prod,lbr_prod_pctchg,lbr_output,lbr_output_pctchg,emp_trk_trans,emp_cour_msg,emp_drv_sls,est_total,regist,freight_val,...,fuel_avg_per_unit,fuel_avg_mpg,fatalities,fatality_rate_permi,fatality_rate_pertruck,vehicle_inv_permi,vehicle_inv_pertruck,miles_per_emp,miles_per_est,emp_per_est
lbr_prod,1.000000,0.088653,0.993344,0.107638,0.705874,0.700313,0.461892,0.835909,0.919242,0.939835,...,-0.671799,0.597159,0.405847,-0.594581,-0.698941,-0.757771,-0.817166,0.376718,0.653261,-0.859579
lbr_prod_pctchg,0.088653,1.000000,0.056056,0.951074,-0.133521,-0.014597,-0.156394,-0.201713,-0.011801,0.025519,...,0.030228,-0.063679,-0.011432,0.052281,0.082117,0.052475,0.081672,0.098444,0.044368,-0.017111
lbr_output,0.993344,0.056056,1.000000,0.085500,0.707328,0.664864,0.419735,0.839772,0.919082,0.955963,...,-0.632547,0.600412,0.379259,-0.632285,-0.720472,-0.791350,-0.836376,0.420454,0.677214,-0.861883
lbr_output_pctchg,0.107638,0.951074,0.085500,1.000000,-0.132887,-0.054735,-0.184988,-0.157473,-0.034987,-0.448476,...,0.110600,-0.037543,-0.019109,0.013349,0.066455,0.035380,0.076445,0.040685,0.047893,0.039406
emp_trk_trans,0.705874,-0.133521,0.707328,-0.132887,1.000000,0.773193,0.893084,0.907816,0.663253,0.687130,...,-0.381377,0.255408,0.704852,-0.241437,-0.284492,-0.420252,-0.425770,-0.069084,0.233375,-0.371789
emp_cour_msg,0.700313,-0.014597,0.664864,-0.054735,0.773193,1.000000,0.789986,0.906925,0.704046,0.498809,...,-0.671424,0.431540,0.597472,-0.216260,-0.319471,-0.340507,-0.405917,0.058335,0.193486,-0.528807
emp_drv_sls,0.461892,-0.156394,0.419735,-0.184988,0.893084,0.789986,1.000000,0.895897,0.439315,0.049709,...,-0.504121,0.199388,0.867637,0.506594,0.367880,0.343080,0.233576,-0.243179,-0.334054,-0.109878
est_total,0.835909,-0.201713,0.839772,-0.157473,0.907816,0.906925,0.895897,1.000000,0.813994,0.692921,...,0.417154,0.724669,-0.223507,-0.758980,-0.773108,-0.801980,-0.735997,0.023180,0.632946,-0.539477
regist,0.919242,-0.011801,0.919082,-0.034987,0.663253,0.704046,0.439315,0.813994,1.000000,0.954997,...,-0.075900,0.725010,-0.037612,-0.561814,-0.713140,-0.779739,-0.913235,0.693731,0.850378,-0.832893
freight_val,0.939835,0.025519,0.955963,-0.448476,0.687130,0.498809,0.049709,0.692921,0.954997,1.000000,...,-0.701849,0.485919,0.519792,-0.793770,-0.950924,-0.923391,-0.972356,0.812604,0.869147,-0.994634


In [ ]:
df2 = df.loc[1987:2020].interpolate().fillna(method = 'bfill')

In [ ]:
#select features using recursive feature elimination (RFE)

X = df2.iloc[:, 1:]
y = df2.iloc[:, 0]

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=10)
selector = selector.fit(X, y)
print(X.columns[selector.support_])

Index(['lbr_prod_pctchg', 'freight_val_pctchg', 'freight_val_pcttotal',
       'freight_tons_pctchg', 'freight_tons_pcttotal',
       'freight_tonmi_pcttotal', 'cptm', 'fatality_rate_permi',
       'fatality_rate_pertruck', 'vehicle_inv_permi'],
      dtype='object')


In [ ]:
#create new dataframe with 'lbr_prod' as the target variable and (4) features. 'freight_val_pctchg','freight_tonmi_pcttotal', 'cptm', 'fatality_rate_permi' will be used to eliminate redundancy and multicollinearity.

df3 = df2[['lbr_prod', 'freight_val_pctchg','freight_tonmi_pcttotal', 'cptm', 'fatality_rate_permi']].loc[1997:2018]

In [ ]:
df3

,lbr_prod,freight_val_pctchg,freight_tonmi_pcttotal,cptm,fatality_rate_permi
year,,,,,
1997,86.0,0.129326,0.674276,0.0997,0.38
1998,86.0,0.154207,0.678996,0.1017,0.38
1999,86.0,0.179088,0.683716,0.1056,0.37
2000,88.0,0.203968,0.688436,0.1112,0.37
2001,89.0,0.228849,0.693157,0.1076,0.34
2002,92.0,0.253730,0.697877,0.1003,0.32
2003,95.0,0.271087,0.689827,0.1012,0.33
2004,94.0,0.288444,0.681777,0.1088,0.35
2005,95.0,0.305802,0.673728,0.1219,0.36


#OLS model

In [ ]:
X = df3.iloc[:, 1:]
y = df3.iloc[:, 0]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               lbr_prod   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     19.73
Date:                Wed, 15 Mar 2023   Prob (F-statistic):           3.28e-06
Time:                        15:17:29   Log-Likelihood:                -57.154
No. Observations:                  22   AIC:                             124.3
Df Residuals:                      17   BIC:                             129.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -13

In [ ]:
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 10.568920591615692


In [ ]:
print ("df = pd.DataFrame( %s )" % (str(df3.to_dict())))

df = pd.DataFrame( {'lbr_prod': {1997: 86.0, 1998: 86.0, 1999: 86.0, 2000: 88.0, 2001: 89.0, 2002: 92.0, 2003: 95.0, 2004: 94.0, 2005: 95.0, 2006: 95.0, 2007: 96.0, 2008: 96.0, 2009: 90.0, 2010: 101.0, 2011: 103.0, 2012: 100.0, 2013: 101.0, 2014: 105.0, 2015: 106.0, 2016: 105.0, 2017: 111.0, 2018: 112.0}, 'freight_val_pctchg': {1997: 0.1293260473588342, 1998: 0.15420680562900282, 1999: 0.17908756389917146, 2000: 0.20396832216934008, 2001: 0.2288490804395087, 2002: 0.2537298387096773, 2003: 0.2710871112990114, 2004: 0.28844438388834553, 2005: 0.30580165647767965, 2006: 0.32315892906701377, 2007: 0.3405162016563479, 2008: 0.31551516861874435, 2009: 0.29051413558114075, 2010: 0.2655131025435372, 2011: 0.24051206950593368, 2012: 0.2155110364683301, 2013: 0.17766913018626043, 2014: 0.13982722390419075, 2015: 0.10198531762212107, 2016: 0.06414341134005139, 2017: 0.0263015050579817, 2018: 0.0263015050579817}, 'freight_tonmi_pcttotal': {1997: 0.6742758361518226, 1998: 0.6789960035294887, 1999:

#Lasso

##LASSO trial 1

In [ ]:
X = df3.iloc[:, 1:]
y = df3.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
lasso_model = LassoCV(cv=5, random_state=0)
lasso_model.fit(X_train, y_train)
y_pred = lasso_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
n = len(y_test)
p = X.shape[1]
adjusted_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))

print("R-squared:", r_squared)
print("Adjusted R-squared:", adjusted_r_squared)
print("Mean squared error:", mse)
print(lasso_model.coef_)

R-squared: 0.46686773894962075
Adjusted R-squared: -inf
Mean squared error: 22.732759611188172
[-2.12837376 -0.          5.01421888  0.        ]


<ipython-input-11-02019b077006>:14: RuntimeWarning: divide by zero encountered in double_scalars
  adjusted_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))


##LASSO trial 2

In [ ]:
lasso_model = LassoCV(cv=5, random_state=0, max_iter=10000, alphas=[0.001, 0.01, 0.1, 1, 10, 100])
lasso_model.fit(X_train, y_train)
y_pred = lasso_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
adj_r_squared = 1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("R-squared:", r_squared)
print("Adjusted R-squared:", adjusted_r_squared)
print("Mean squared error:", mse)
print(lasso_model.coef_)

R-squared: 0.4604425580600703
Adjusted R-squared: -inf
Mean squared error: 23.006729324318602
[-1.81513638 -0.          4.70100949 -0.        ]


<ipython-input-12-e4cb4e1fab84>:6: RuntimeWarning: divide by zero encountered in double_scalars
  adj_r_squared = 1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)


#Neural Network

##NN Trial 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:, 1:], df3.iloc[:, 0], test_size=0.2, random_state=42)

# Scale the input variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=4, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=16, validation_split=0.2)

# Evaluate the model on the test set
mse, mae = model.evaluate(X_test_scaled, y_test)

y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
n = X_test_scaled.shape[0]  # Number of samples
p = X_test_scaled.shape[1]  # Number of features
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print('R-squared:', r2)
print('Adjusted R-squared:', adj_r2)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 9455.2432 - mae: 96.8775 - val_loss: 9862.8467 - val_mae: 99.2236
Epoch 2/100
1/1 [==============================] - 0s 44ms/step - loss: 9443.3818 - mae: 96.8160 - val_loss: 9850.6650 - val_mae: 99.1623
Epoch 3/100
1/1 [==============================] - 0s 75ms/step - loss: 9431.2461 - mae: 96.7530 - val_loss: 9838.6602 - val_mae: 99.1019
Epoch 4/100
1/1 [==============================] - 0s 43ms/step - loss: 9418.9062 - mae: 96.6889 - val_loss: 9826.5830 - val_mae: 99.0411
Epoch 5/100
1/1 [==============================] - 0s 67ms/step - loss: 9406.4805 - mae: 96.6242 - val_loss: 9814.2471 - val_mae: 98.9789
Epoch 6/100
1/1 [==============================] - 0s 50ms/step - loss: 9394.0098 - mae: 96.5593 - val_loss: 9801.6562 - val_mae: 98.9154
Epoch 7/100
1/1 [==============================] - 0s 47ms/step - loss: 9381.3750 - mae: 96.4936 - val_loss: 9788.7598 - val_mae: 98.8503
Epoch 8/100
1/1 [===================

<ipython-input-13-1bbff5f676ac>:27: RuntimeWarning: divide by zero encountered in double_scalars
  adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)


##NN Trial 2

In [ ]:
# Build the neural network model
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

# Compile the model
optimizer = Adam(learning_rate = 0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=500, batch_size=16, validation_split=0.2)

# Evaluate the model on the test set
mse, mae = model.evaluate(X_test_scaled, y_test)

y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
n = X_test_scaled.shape[0]  # Number of samples
p = X_test_scaled.shape[1]  # Number of features
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print('R-squared:', r2)
print('Adjusted R-squared:', adj_r2)

Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 9578.5645 - mae: 97.4962 - val_loss: 9992.1025 - val_mae: 99.8650
Epoch 2/500
1/1 [==============================] - 0s 27ms/step - loss: 9571.2363 - mae: 97.4609 - val_loss: 9976.7949 - val_mae: 99.7892
Epoch 3/500
1/1 [==============================] - 0s 28ms/step - loss: 9562.4180 - mae: 97.4122 - val_loss: 9961.9902 - val_mae: 99.7158
Epoch 4/500
1/1 [==============================] - 0s 29ms/step - loss: 9523.1416 - mae: 97.2184 - val_loss: 9947.7207 - val_mae: 99.6449
Epoch 5/500
1/1 [==============================] - 0s 29ms/step - loss: 9522.0283 - mae: 97.2145 - val_loss: 9933.8760 - val_mae: 99.5760
Epoch 6/500
1/1 [==============================] - 0s 29ms/step - loss: 9503.0254 - mae: 97.1108 - val_loss: 9920.1445 - val_mae: 99.5076
Epoch 7/500
1/1 [==============================] - 0s 28ms/step - loss: 9485.0176 - mae: 97.0236 - val_loss: 9906.3906 - val_mae: 99.4390
Epoch 8/500
1/1 [===================

<ipython-input-14-d6ee8bdd2c46>:23: RuntimeWarning: divide by zero encountered in double_scalars
  adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)


##NN Trial 3

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Use a different optimizer and compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Use early stopping during training
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# Train the model with more epochs and use early stopping
history = model.fit(X_train_scaled, y_train, epochs=200, batch_size=16, validation_split=0.2, callbacks=[early_stop])

# Evaluate the model on the test set
mse, mae = model.evaluate(X_test_scaled, y_test)

y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
n = X_test_scaled.shape[0]  # Number of samples
p = X_test_scaled.shape[1]  # Number of features
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print('R-squared:', r2)
print('Adjusted R-squared:', adj_r2)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 9533.6338 - mae: 97.2640 - val_loss: 9952.1328 - val_mae: 99.6662
Epoch 2/200
1/1 [==============================] - 0s 39ms/step - loss: 9487.5352 - mae: 97.0456 - val_loss: 9944.0186 - val_mae: 99.6256
Epoch 3/200
1/1 [==============================] - 0s 57ms/step - loss: 9462.3291 - mae: 96.9048 - val_loss: 9935.6602 - val_mae: 99.5836
Epoch 4/200
1/1 [==============================] - 0s 36ms/step - loss: 9513.2988 - mae: 97.1740 - val_loss: 9927.9912 - val_mae: 99.5452
Epoch 5/200
1/1 [==============================] - 0s 58ms/step - loss: 9499.8027 - mae: 97.0895 - val_loss: 9920.5273 - val_mae: 99.5078
Epoch 6/200
1/1 [==============================] - 0s 40ms/step - loss: 9477.6182 - mae: 96.9772 - val_loss: 9913.2207 - val_mae: 99.4711
Epoch 7/200
1/1 [==============================] - 0s 36ms/step - loss: 9490.1006 - mae: 97.0309 - val_loss: 9905.8730 - val_mae: 99.4342
Epoch 8/200
1/1 [===================

<ipython-input-15-2bd1b7181ec7>:27: RuntimeWarning: divide by zero encountered in double_scalars
  adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)


##NN Trial 4

In [ ]:
model = Sequential([
    Dense(64, input_dim=4, activation='relu'),
    Dropout(0.2),  # Add dropout layer
    Dense(32, activation='relu'),
    Dropout(0.2),  # Add dropout layer
    Dense(16, activation='relu'),  # Add additional hidden layer
    Dense(1, activation='linear')
])

# Define the optimizer and compile the model
optimizer = Adam(lr=0.0001)  # Adjust learning rate
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=16, validation_split=0.2)

# Evaluate the model on the test set
mse, mae = model.evaluate(X_test_scaled, y_test)

y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
n = X_test_scaled.shape[0]  # Number of samples
p = X_test_scaled.shape[1]  # Number of features
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print('R-squared:', r2)
print('Adjusted R-squared:', adj_r2)

Epoch 1/100
1/1 [==============================] - 1s 807ms/step - loss: 9522.0146 - mae: 97.1980 - val_loss: 9959.1182 - val_mae: 99.6956
Epoch 2/100
1/1 [==============================] - 0s 34ms/step - loss: 9502.8428 - mae: 97.1027 - val_loss: 9953.3408 - val_mae: 99.6665
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 9491.3916 - mae: 97.0368 - val_loss: 9947.4121 - val_mae: 99.6366
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 9502.7568 - mae: 97.1073 - val_loss: 9941.1523 - val_mae: 99.6051
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 9482.1494 - mae: 96.9926 - val_loss: 9934.7734 - val_mae: 99.5731
Epoch 6/100
1/1 [==============================] - 0s 39ms/step - loss: 9469.5273 - mae: 96.9299 - val_loss: 9928.2988 - val_mae: 99.5405
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 9469.8994 - mae: 96.9325 - val_loss: 9921.8418 - val_mae: 99.5079
Epoch 8/100
1/1 [================

<ipython-input-16-78ce236eb788>:24: RuntimeWarning: divide by zero encountered in double_scalars
  adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)


Overall, the LASSO model produces the best results, and the OLS model also performs quite well. In this case, OLS could be used over LASSO due to simplicity. 

The neural network model variations all do not perform well at all. This could possibly be due to the limited data size. 